In [ ]:
%use lets-plot
%use plotly
import kotlinx.coroutines.*

val data = runBlocking { readAllUData("./log.anon") }

In [ ]:
val queryPerSecond = mutableListOf<Pair<UInt, List<UData>>>()
var current = mutableListOf<UData>()
var currentTime = data.first().unixTimeStamp
data.forEach { u ->
    if (u.unixTimeStamp == currentTime) {
        current.add(u)
    } else {
        queryPerSecond.add(Pair(currentTime, current))
        for (i in (currentTime + 1u)..(u.unixTimeStamp - 1u)) {
            // Empty list to add 0 values
            queryPerSecond.add(Pair(i, listOf()))
        }
        current = mutableListOf(u)
        currentTime = u.unixTimeStamp
    }
}

val amountOfSecondsWithoutValidations =
    queryPerSecond.last().first - queryPerSecond.first().first - (queryPerSecond.size.toUInt() - 1u)
if (amountOfSecondsWithoutValidations != 0u) {
    println("Error: $amountOfSecondsWithoutValidations seconds without validations.")
}

In [ ]:
import java.time.Instant
import java.time.OffsetDateTime
import java.time.ZoneOffset
import kotlin.system.exitProcess

val times = queryPerSecond.map { t ->
    val time = t.first.toLong()
    Instant.ofEpochSecond(time)
}
val amount = queryPerSecond.map { t -> t.second.size }

val plotTime = mapOf<String, List<Any>>(
    "time" to times,
    "values" to amount
)

val plot = ggplot(plotTime) +
        geomLine { x = "time"; y = "values" } +
        ggtitle("Amount of validations over time") +
        ggsize(4000, 500)
plot.show()

In [ ]:
val timeString = times.map { t -> t.toString() }

fun plotOverTime(y: List<Int>, title: String): Plot {
    return Plotly.plot {
        scatter {
            x(*timeString.toTypedArray())
            y(*y.toTypedArray())
        }

        layout {
            title {
                text = "\$\\text{$title}$"
            }
        }
    }
}

plotOverTime(amount, "Amount of validations over time")



In [ ]:
val amountRootZone =
    queryPerSecond.map { (time, data) -> Pair(time, data.filter { ud -> ud.domainName.size == 1 }) }
        .map { t -> t.second.size }

plotOverTime(amountRootZone, "Amount of validations over time for root zone")

In [ ]:
val amountTLDs =
    queryPerSecond.map { (time, data) -> Pair(time, data.filter { ud -> ud.domainName.size <= 2 }) }
        .map { t -> t.second.size }

plotOverTime(amountTLDs, "Amount of validations over time for TLDs and root zone")